## Scrapy 

Основные команды:

|Команда|Описание|
|--|--|
|`scrapy startproject <name_project>`|создать проект|
|`scrapy genspider <spider_name> <base_url>`|создать паука base_url например tiktok.com|
|`scrapy crawl <spider_name> -o <output_file>`|запустить паука `-o <output_file>` не обязательный параметр(например out.json)|
|||


### Полезные настройки setting.py

```python
DOWNLOAD_DELAY = 0.25 # Задержка в секундах
RANDOMIZE_DOWNLOAD_DELAY = True # Рандомизация задержек от 0.5 до 1.5
LOG_FILE = 'logging.txt' # Название файла для ведения логов, изначально None
LOG_LEVEL = 'CRITICAL, ERROR, WARNING, INFO, DEBUG' # Какие ошибки выводяться
LOG_STDOUT = True # Позволяет пользоваться print(), по умолчанию False

### Установить рандомного user-agent

В `settings.py` расскоментировать:
```python 
DOWNLOADER_MIDDLEWARES = {
    'mwaredemo.middlewares.MwarendemoDownloaderMiddleware': 400,
}
```
в файле `middlewares.py` создать свой класс:
```python
import random
from .user_agents_firefox import USERAGENTS # Ну или еще откуда подгрузить

class RotateUserAgentMiddleware:
    def process_request(self, request, spider):
        user_agent = random.choice(USERAGENTS)
        request.headers['User-Agent'] = user_agent

```
Затем добавляем созданный класс в `settings.py`
```python
DOWNLOADER_MIDDLEWARES = {
    # Цифры указывают на порядок выполнения
    'example.middlewares.RotateUserAgentMiddleware': 300, 
    'example.middlewares.MwarendemoDownloaderMiddleware': 400,
}

### Прикольные функции

Для подсказок добавить аннотацию к response
```python
from scrapy.http.response.html import HtmlResponse

class MySpider:
    ...
    def parse(response: HtmlResponse):
        pass

Можно установить в паука:
```python
def parse(self, response):
    self.logger.info(f'Спарсил страницу: {response.url}') 
    self.logger.warning(f'Не спарсил {response.url}')

### Вариант установки настроек

- В файле `settings.py`:
```python
DOWNLOADER_MIDDLEWARES = {
    'example.setting': None,
}
```
---
- Непосредственно в пауке
```python
class Myspider(scrapy.Spider):
    name = 'example_name'
    custom_settings = {
        'DOWNLOADER_MIDDLEWARES' : {
            'example.setting': None,
        }
    }

## Scrapy & Playwright

```bash
pip install scrapy-playwright
```
Также необходимо добавить параметры в `settings.py` см [документацию](https://github.com/scrapy-plugins/scrapy-playwright)

## Playwright

```bash
pip install playwright
playwright install

### Синхронный 

```python
from playwright.sync_api import sync_playwright

# Делаем с with
with sync_playwright() as p:
    # Создаем браузер
    browser = p.firefox.launch()
    # Создаем страницу
    page = browser.new_page()
    # Переходим
    page.goto('https://www.ozon.ru/seller/87879/products')
    # Ждем(можно поставить задержку из разряда пока-не-прогрузиться определнный элемент)
    page.wait_for_timeout(2000)
    # Достаем инфо по Xpath / можно использовать query_selector_all()
    company = page.query_selector('//h1')
    # Не забываем закрывать браузер
    browser.close()

# Можно дополнительно: обратиться ко всему контенту
page.content